In [ ]:
sys_path = r'C:\Users\sgala\PycharmProjects\OCR\OCR' # path to project
path_to_tesstrain_code = r"C:/Users/sgala/Documents/python_projects/tesseract/tesstrain" #download from https://github.com/tesseract-ocr/tesstrain/tree/main and follow instructions
#recomended - watch: https://www.youtube.com/watch?v=SvhoBT-PnME&ab_channel=SL7Tech
relative_path_to_starting_model = '../tessdata/' #default tesstrain data folder
tesseract_exe_path = r"C:\Program Files\Tesseract-OCR\tesseract.exe" #install tesseract from: https://github.com/UB-Mannheim/tesseract/wiki
tessdata_path = r"C:\Program Files\Tesseract-OCR\tessdata\script" #in tesseract folder after install
bash_path = r"C:/Program Files/Git/bin/bash.exe" #path to a bash on your computer (problem in windows mostly)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import shutil
import sys
sys.path += [sys_path]
from generate_pdfs.create_pdf_dataset import create_pdf_dataset
from tesseract.prepare_train_data import pdf_to_train_images
from tesseract.prepare_validation_data import process_pdf_to_images_and_data
from tesseract.metrics import evaluate_ocr_batch
from tesseract.match_gt_pred import match_predictions_to_gt
from tesseract.pipeline.preprocess_funcs import align_text,enhance_soft_binarization,resize_image_constant_median_size,preprocess,preprocessConfig
from tesseract.pipeline.postprocess_funcs import postprocess_ocr_data,visualize_ocr
import pytesseract
import pandas as pd
import os
import cv2
import subprocess
import numpy as np
import matplotlib.pyplot as plt

pytesseract.pytesseract.tesseract_cmd = tesseract_exe_path
os.environ["TESSDATA_PREFIX"] = tessdata_path

In [ ]:
# def align_text(image: np.ndarray) -> np.ndarray:
#     """
#     Corrects the skew of text in an image by detecting and rotating it to the correct orientation.

#     Uses the top 10% of contours and aggregates their angles to compute the best rotation.

#     :param image: Input binary image (output of enhance_image)
#     :return: Deskewed image as a NumPy array
#     """
#     # Invert colors for contour detection (white text on black background)
#     inverted = cv2.bitwise_not(image)

#     # Detect edges for better contour extraction
#     edges = cv2.Canny(inverted, 50, 150)
#     plt.figure(figsize=(24, 16))
#     plt.imshow(edges)
#     plt.show()
#     # Find contours
#     contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#     if not contours:
#         print("No contours found for alignment.")
#         return image

#     # Sort contours by area and select the top 10%
#     contours = sorted(contours, key=cv2.contourArea, reverse=True)
#     top_contours = contours[:max(1, len(contours) // 10)]
#     top_contour_img = np.zeros_like(image)
#     for cont in top_contours:
#         for c in cont:
#             top_contour_img[c[0][1],c[0][0]] = 255
    
    
#     plt.figure(figsize=(24, 16))
#     plt.imshow(cv2.dilate(top_contour_img,np.ones((5,5))))
#     plt.show()
#     angles = []

#     for contour in top_contours:
#         rect = cv2.minAreaRect(contour)
#         print(rect)
#         angle = rect[-1]
#         angle = (angle + 45)%90 - 45
#         angles.append(angle)
    
#     # Compute the median angle to avoid outliers
#     print(angles)
#     if angles:
#         median_angle = np.median(angles)
#     else:
#         median_angle = 0
#     print(angle)

#     # Get image dimensions
#     (h, w) = image.shape[:2]
#     center = (w // 2, h // 2)

#     # Perform the rotation
#     rotation_matrix = cv2.getRotationMatrix2D(center, median_angle, 1.0)
#     deskewed = cv2.warpAffine(image, rotation_matrix, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)

#     return deskewed

In [ ]:
image_path = r"C:\Users\sgala\Downloads\WhatsApp Image 2025-02-06 at 15.53.09.jpeg"
image_orig = cv2.imread(image_path)

In [ ]:
config = preprocessConfig()


In [ ]:
from itertools import product

In [ ]:
for perform_stages in product([False,True],repeat=3):
    config.resize = perform_stages[0]
    config.enhance = perform_stages[1]
    config.align = perform_stages[2]
    preprocessed_image = preprocess(image_orig,config)
    cv2.imwrite("temp.png",preprocessed_image)
    image_data_pred = pytesseract.image_to_data("temp.png" ,output_type=pytesseract.Output.DICT,lang="heb_random_top_five_hundred_two")
    pred_bbox,pred_text,pred_conf = postprocess_ocr_data(image_data_pred,min_confidence=0)
    wers = [0]*len(pred_bbox)
    image_with_ocr = visualize_ocr(preprocessed_image,pred_bbox,pred_text,wers,font_size=int(0.007*preprocessed_image.shape[0]),thickness=int(0.001*preprocessed_image.shape[0])+1)
    plt.figure(figsize=(36, 24))
    plt.imshow(image_with_ocr)
    plt.axis('off')
    plt.title(perform_stages)
    plt.show()

